# **Market Analysis &ndash; Part 1**

This Notebook performs the computations and visualizations for the first part of the market analysis (customer preferences and issues) of our paper:

* Create sample of real product reviews that are about T-shirts
* Process the data from the manual market analysis
* Analysis of the manual market analysis
* Visualization of the results

## Preparation

In [1]:
# Import the necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from math import sqrt

## Load the data

In [ ]:
# Load the dataset
df = pd.read_parquet('Data/preprocessed_reviews.parquet')

# Display the first few rows
df.head()

In [ ]:
# Use a regex pattern to filter rows where any variation of "T-shirt" or "shirt" appears in either the 'title'

# Create a mask for the condition where any variation of "T-shirt" or "shirt" appears
mask = df['title'].str.contains(r'\bt[-]?shirt\b', case=False, na=False) | \
        df['title'].str.contains(r'\bshirt\b', case=False, na=False) 

# Filter the DataFrame based on the mask
filtered_df = df[mask]

# Display the filtered DataFrame
filtered_df.head()

In [ ]:
# Extract a 100 review sample from the real reviews for the market analysis 
sampled_text = filtered_df['text'].sample(n=100, random_state=42)

# Create a DataFrame with the index as the first column and the review as the second column
sampled_df = pd.DataFrame({
    'Index old': sampled_text.index,
    'Review': sampled_text.values
})

# Display the sampled DataFrame
sampled_df

In [ ]:
# Save as CSV file
sampled_df.to_csv('real_reviews_tshirt_sample.csv', index=False)

## Load the data

In [ ]:
# Load the coded reviews from the different excel sheets
df_gpt2_rater1 = pd.read_excel('Data/market_analysis_rater1.xlsx', sheet_name='GPT-2 Market Analysis', dtype=str)
df_llama_rater1 = pd.read_excel('Data/market_analysis_rater1.xlsx', sheet_name='LLaMA 3 Market Analysis', dtype=str)
df_real_rater1 = pd.read_excel('Data/market_analysis_rater1.xlsx', sheet_name='Real Reviews Market Analysis', dtype=str)

df_gpt2_rater2 = pd.read_excel('Data/market_analysis_rater2.xlsx', sheet_name='GPT-2 Market Analysis', dtype=str)
df_llama_rater2 = pd.read_excel('Data/market_analysis_rater2.xlsx', sheet_name='LLaMA 3 Market Analysis', dtype=str)
df_real_rater2 = pd.read_excel('Data/market_analysis_rater2.xlsx', sheet_name='Real Reviews Market Analysis', dtype=str)

# Check one of the DataFrames
df_gpt2_rater1.head()

## Data Processing

In [ ]:
# Define a function to process each DataFrame
def process_dataframe(df):
    # Combine the columns 'Code 1' to 'Code 9' into a new column 'Combined Code'
    df['Combined Code'] = df[['Code 1', 'Code 2', 'Code 3', 'Code 4', 'Code 5', 'Code 6', 'Code 7', 'Code 8', 'Code 9']] \
        .apply(lambda row: [code for code in row if pd.notna(code)], axis=1)

    # Drop the 'Code 1' to 'Code 9' columns
    df.drop(columns=[f'Code {i}' for i in range(1, 10)], inplace=True)

    return df

# List of all DataFrames to process
dataframes = [df_gpt2_rater1, df_llama_rater1, df_real_rater1, df_gpt2_rater2, df_llama_rater2, df_real_rater2]

# Apply the function to each DataFrame in the list
for df in dataframes:
    process_dataframe(df)

In [ ]:
# Define a function to normalize and compare DataFrames
def create_comparison_df(df_rater1, df_rater2):
    # Normalize 'Combined Code' by converting each list to a sorted tuple
    df_rater1['Normalized Code'] = df_rater1['Combined Code'].apply(lambda x: tuple(sorted(x)))
    df_rater2['Normalized Code'] = df_rater2['Combined Code'].apply(lambda x: tuple(sorted(x)))

    # Create a new DataFrame with 'Review', 'Rater1', 'Rater2', and 'Comparison' columns
    comparison_df = pd.DataFrame({
        'Review': df_rater1['Review'],
        'Rater1': df_rater2['Combined Code'],
        'Rater2': df_rater2['Combined Code'],
        'Comparison': df_rater1['Normalized Code'] == df_rater2['Normalized Code']
    })

    return comparison_df

# Create the comparison DataFrames for each pair
df_gpt2 = create_comparison_df(df_gpt2_rater1, df_gpt2_rater2)
df_llama = create_comparison_df(df_llama_rater1, df_llama_rater2)
df_real = create_comparison_df(df_real_rater1, df_real_rater2)

In [ ]:
# Define a function to calculate code frequencies for all possible categories 
def calculate_code_frequencies(df):
    # Combine all lists from Rater 1 and Rater 2 columns into one Series
    all_codes = pd.Series(df['Rater1'].sum() + df['Rater2'].sum())
    
    # Use value_counts() to get the frequency of each code
    code_frequencies = all_codes.value_counts().reset_index()
    
    # Rename the columns for better understanding
    code_frequencies.columns = ['Code', 'Frequency']
    
    return code_frequencies

# Calculate code frequencies for each DataFrame
code_frequencies_gpt2 = calculate_code_frequencies(df_gpt2)
code_frequencies_llama = calculate_code_frequencies(df_llama)
code_frequencies_real = calculate_code_frequencies(df_real)

## Cosine similarity

In [ ]:
# Merge DataFrames on Code to align frequencies
merged_df = pd.merge(code_frequencies_real, code_frequencies_gpt2, on='Code', how='outer', suffixes=('_df1', '_df2'))

# Replace NaN with 0 for frequencies
merged_df.fillna(0, inplace=True)

# Convert frequencies to integers
merged_df['Frequency_df1'] = merged_df['Frequency_df1'].astype(int)
merged_df['Frequency_df2'] = merged_df['Frequency_df2'].astype(int)

# Calculate dot product
dot_product = sum(merged_df['Frequency_df1'] * merged_df['Frequency_df2'])

# Calculate magnitudes
magnitude1 = sqrt(sum(merged_df['Frequency_df1'] ** 2))
magnitude2 = sqrt(sum(merged_df['Frequency_df2'] ** 2))

# Calculate cosine similarity
cosine_similarity = dot_product / (magnitude1 * magnitude2)
cosine_similarity

In [ ]:
# Merge DataFrames on Code to align frequencies
merged_df = pd.merge(code_frequencies_real, code_frequencies_llama, on='Code', how='outer', suffixes=('_df1', '_df2'))

# Replace NaN with 0 for frequencies
merged_df.fillna(0, inplace=True)

# Convert frequencies to integers
merged_df['Frequency_df1'] = merged_df['Frequency_df1'].astype(int)
merged_df['Frequency_df2'] = merged_df['Frequency_df2'].astype(int)

# Calculate dot product
dot_product = sum(merged_df['Frequency_df1'] * merged_df['Frequency_df2'])

# Calculate magnitudes
magnitude1 = sqrt(sum(merged_df['Frequency_df1'] ** 2))
magnitude2 = sqrt(sum(merged_df['Frequency_df2'] ** 2))

# Calculate cosine similarity
cosine_similarity = dot_product / (magnitude1 * magnitude2)
cosine_similarity

In [ ]:
# Merge DataFrames on Code to align frequencies
merged_df = pd.merge(code_frequencies_gpt2, code_frequencies_llama, on='Code', how='outer', suffixes=('_df1', '_df2'))

# Replace NaN with 0 for frequencies
merged_df.fillna(0, inplace=True)

# Convert frequencies to integers
merged_df['Frequency_df1'] = merged_df['Frequency_df1'].astype(int)
merged_df['Frequency_df2'] = merged_df['Frequency_df2'].astype(int)

# Calculate dot product
dot_product = sum(merged_df['Frequency_df1'] * merged_df['Frequency_df2'])

# Calculate magnitudes
magnitude1 = sqrt(sum(merged_df['Frequency_df1'] ** 2))
magnitude2 = sqrt(sum(merged_df['Frequency_df2'] ** 2))

# Calculate cosine similarity
cosine_similarity = dot_product / (magnitude1 * magnitude2)
cosine_similarity

## Check the top categories

In [ ]:
# Filter the top ten codes by frequency
code_frequencies_gpt2_topten = code_frequencies_gpt2.nlargest(10, 'Frequency')
code_frequencies_llama_topten = code_frequencies_llama.nlargest(10, 'Frequency')
code_frequencies_real_topten = code_frequencies_real.nlargest(10, 'Frequency')

# Display the resulting DataFrame with the top ten codes
code_frequencies_gpt2_topten

In [ ]:
# Use just the main categories

# Define a function to calculate code frequencies with modified strings
def calculate_code_frequencies_main_categories(df):
    # Combine all lists from 'Rater1' and 'Rater2' columns into one Series
    all_codes = pd.Series(df['Rater1'].sum() + df['Rater2'].sum())
    
    # Modify strings: use first 2 characters for lengths 2 or 4, first 3 for lengths 3 or 5
    modified_codes = all_codes.apply(lambda x: x[:2] if len(x) in [2, 4] else x[:3])

    # Use value_counts() to get the frequency of each code
    code_frequencies = modified_codes.value_counts().reset_index()
    
    # Rename the columns for better understanding
    code_frequencies.columns = ['Code', 'Frequency']
    
    return code_frequencies

# Calculate code frequencies for each DataFrame
code_frequencies_gpt2_main = calculate_code_frequencies_main_categories(df_gpt2)
code_frequencies_llama_main = calculate_code_frequencies_main_categories(df_llama)
code_frequencies_real_main = calculate_code_frequencies_main_categories(df_real)

In [ ]:
# Filter the top ten main codes by frequency
code_frequencies_gpt2_topten_main = code_frequencies_gpt2_main.nlargest(10, 'Frequency')
code_frequencies_llama_topten_main = code_frequencies_llama_main.nlargest(10, 'Frequency')
code_frequencies_real_topten_main = code_frequencies_real_main.nlargest(10, 'Frequency')

# Display the resulting DataFrame with the top ten codes
code_frequencies_gpt2_topten_main

## Visualizations

In [ ]:
# Fixed layout parameters
left_margin = 0.1
right_margin = 0.9
top_margin = 0.8
bottom_margin = 0.4

In [ ]:
# Plot GPT-2 all categories
plt.figure(figsize=(10, 6))
# Define custom colors using valid color names or hex codes
custom_colors = ['steelblue', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'steelblue', 'steelblue']

sns_barplot = sns.barplot(x='Code', y='Frequency', data=code_frequencies_gpt2_topten, palette=custom_colors)
for bar in sns_barplot.patches:
    bar.set_edgecolor('black')  # Set the border color
    bar.set_linewidth(1)        # Set the border thickness

# Manually add custom labels to the bars
custom_labels = ['Sizing\n(negative)', 'Soft\n(Material/Fabric)', 'Fit\n(positive)', 'Comfortable', 'Quality\n(positive)', 'Color\n(positive)', 'Price\n(positive)', 'Style/Design\n(positive)', 'Thin\n(Material/Fabric)\n(negative)', 'Fit\n(negative)']
plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=10, rotation=30, ha='right')

# Add custom legend
legend_elements = [Patch(facecolor='steelblue', edgecolor='black', label='Issues'), Patch(facecolor='orange', edgecolor='black', label='Preferences')]
plt.legend(handles=legend_elements, loc='upper right', fontsize=16)

plt.xlabel('Code', weight='bold', fontsize=18)
plt.ylabel('Frequency', weight='bold', fontsize=18)
plt.yticks(fontsize=15)

# Manually adjust the layout
#plt.subplots_adjust(left=left_margin, right=right_margin, top=top_margin, bottom=bottom_margin)

plt.savefig('gpt-2_all_codes.pdf', format='pdf', dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
# Plot GPT-2 main categories
plt.figure(figsize=(10, 6))
# Define custom colors using valid color names or hex codes
custom_colors = ['orange', 'steelblue', 'steelblue', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'steelblue'] 

sns_barplot = sns.barplot(x='Code', y='Frequency', data=code_frequencies_gpt2_topten_main, palette=custom_colors)
for bar in sns_barplot.patches:
    bar.set_edgecolor('black')  # Set the border color
    bar.set_linewidth(1)        # Set the border thickness

# Manually add custom labels to the bars
custom_labels = ['Material/Fabric\n(positive)', 'Material/Fabric\n(negative)', 'Sizing\n(negative)', 'Fit\n(positive)', 'Comfortable', 'Quality\n(positive)', 'Color\n(positive)', 'Price\n(positive)', 'Style/Design\n(positive)', 'Fit\n(negative)']
plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=10, rotation=30, ha='right')

# Add custom legend
legend_elements = [Patch(facecolor='steelblue', edgecolor='black', label='Issues'), Patch(facecolor='orange', edgecolor='black', label='Preferences')]
plt.legend(handles=legend_elements, loc='upper right', fontsize=16)

plt.xlabel('Code', weight='bold', fontsize=18)
plt.ylabel('Frequency', weight='bold', fontsize=18)
plt.yticks(fontsize=15)

# Manually adjust the layout
#plt.subplots_adjust(left=left_margin, right=right_margin, top=top_margin, bottom=bottom_margin)

plt.savefig('gpt-2_main_codes.pdf', format='pdf', dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
# Plot LLaMA 3 all categories
plt.figure(figsize=(10, 6))
# Define custom colors using valid color names or hex codes
custom_colors = ['orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'steelblue', 'orange']

sns_barplot = sns.barplot(x='Code', y='Frequency', data=code_frequencies_llama_topten, palette=custom_colors)
for bar in sns_barplot.patches:
    bar.set_edgecolor('black')  # Set the border color
    bar.set_linewidth(1)        # Set the border thickness

# Manually add custom labels to the bars
custom_labels = ['Soft\n(Material/Fabric)', 'Comfortable', 'Fit\n(positive)', 'Style/Design\n(positive)', 'Breathable\n(Material/Fabric)', 'Washing behavior\n(positive)', 'Color\n(positive)', 'High quality\n(Material/Fabric)', 'Seams\n(negative)', 'Quality\n(positive)']
plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=10, rotation=30, ha='right')

# Add custom legend
legend_elements = [Patch(facecolor='steelblue', edgecolor='black', label='Issues'), Patch(facecolor='orange', edgecolor='black', label='Preferences')]
plt.legend(handles=legend_elements, loc='upper right', fontsize=16)

plt.xlabel('Code', weight='bold', fontsize=18)
plt.ylabel('Frequency', weight='bold', fontsize=18)
plt.yticks(fontsize=15)

# Manually adjust the layout
#plt.subplots_adjust(left=left_margin, right=right_margin, top=top_margin, bottom=bottom_margin)

plt.savefig('llama-3_all_codes.pdf', format='pdf', dpi=1500, bbox_inches='tight') 
plt.show()

In [ ]:
# Plot LLaMA 3 main categories 
plt.figure(figsize=(10, 6))
# Define custom colors using valid color names or hex codes
custom_colors = ['orange', 'orange', 'orange', 'orange', 'steelblue', 'orange', 'orange', 'orange', 'steelblue', 'steelblue']

sns_barplot = sns.barplot(x='Code', y='Frequency', data=code_frequencies_llama_topten_main, palette=custom_colors)
for bar in sns_barplot.patches:
    bar.set_edgecolor('black')  # Set the border color
    bar.set_linewidth(1)        # Set the border thickness

# Manually add custom labels to the bars
custom_labels = ['Material/Fabric\n(positive)', 'Comfortable\n(positive)', 'Fit\n(positive)', 'Style/Design\n(positive)', 'Material/Fabric\n(negative)', 'Washing behavior\n(positive)', 'Color\n(positive)', 'Quality\n(positive)', 'Quality\n(negative)', 'Washing behavior\n(negative))']
plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=10, rotation=30, ha='right')

# Add custom legend
legend_elements = [Patch(facecolor='steelblue', edgecolor='black', label='Issues'), Patch(facecolor='orange', edgecolor='black', label='Preferences')]
plt.legend(handles=legend_elements, loc='upper right', fontsize=16)

plt.xlabel('Code', weight='bold', fontsize=18)
plt.ylabel('Frequency', weight='bold', fontsize=18)
plt.yticks(fontsize=15)

# Manually adjust the layout
#plt.subplots_adjust(left=left_margin, right=right_margin, top=top_margin, bottom=bottom_margin)

plt.savefig('llama-3_main_codes.pdf', format='pdf', dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
# Plot real reviews all categories
plt.figure(figsize=(10, 6))
# Define custom colors using valid color names or hex codes
custom_colors = ['orange', 'steelblue', 'steelblue', 'orange', 'orange', 'steelblue', 'steelblue', 'orange', 'orange', 'steelblue']

sns_barplot = sns.barplot(x='Code', y='Frequency', data=code_frequencies_real_topten, palette=custom_colors)
for bar in sns_barplot.patches:
    bar.set_edgecolor('black')  # Set the border color
    bar.set_linewidth(1)        # Set the border thickness

# Manually add custom labels to the bars
custom_labels = ['Fit\n(positive)', 'Description\nonline incorrect', 'Sizing\n(negative)', 'Quality\n(positive)', 'Style/Design\n(positive)', 'Fit\n(negative)', 'Quality\n(negative)', 'Comfortable', 'Material/Fabric\n(positive)', 'Material/Fabric\n(negative)']
plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=10, rotation=30, ha='right')

# Add custom legend
legend_elements = [Patch(facecolor='steelblue', edgecolor='black', label='Issues'), Patch(facecolor='orange', edgecolor='black', label='Preferences')]
plt.legend(handles=legend_elements, loc='upper right', fontsize=16)

plt.xlabel('Code', weight='bold', fontsize=18)
plt.ylabel('Frequency', weight='bold', fontsize=18)
plt.yticks(fontsize=15)

# Manually adjust the layout
#plt.subplots_adjust(left=left_margin, right=right_margin, top=top_margin, bottom=bottom_margin)

plt.savefig('real_all_codes.pdf', format='pdf', dpi=1500, bbox_inches='tight')
plt.show()

In [ ]:
# Plot real reviews main categories 
plt.figure(figsize=(10, 6))
# Define custom colors using valid color names or hex codes
custom_colors = ['orange', 'steelblue', 'steelblue', 'steelblue', 'orange', 'orange', 'steelblue', 'orange', 'steelblue', 'steelblue']

sns_barplot = sns.barplot(x='Code', y='Frequency', data=code_frequencies_real_topten_main, palette=custom_colors)
for bar in sns_barplot.patches:
    bar.set_edgecolor('black')  # Set the border color
    bar.set_linewidth(1)        # Set the border thickness

# Manually add custom labels to the bars
custom_labels = ['Fit\n(positive)', 'Material/Fabric\n(negative)', 'Description online\n(negative)', 'Sizing\n(negative)', 'Material/Fabric\n(positive)', 'Quality\n(positive)', ' Fit\n(negative)', 'Style/Design\n(positive)', 'Length\n(negative)', 'Quality\n(negative)']
plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=10, rotation=30, ha='right')

# Add custom legend
legend_elements = [Patch(facecolor='steelblue', edgecolor='black', label='Issues'), Patch(facecolor='orange', edgecolor='black', label='Preferences')]
plt.legend(handles=legend_elements, loc='upper right', fontsize=16)

plt.xlabel('Code', weight='bold', fontsize=18)
plt.ylabel('Frequency', weight='bold', fontsize=18)
plt.yticks(fontsize=15)

# Manually adjust the layout
#plt.subplots_adjust(left=left_margin, right=right_margin, top=top_margin, bottom=bottom_margin)

plt.savefig('real_main_codes.pdf', format='pdf', dpi=1500, bbox_inches='tight')
plt.show()